In [11]:
import pandas as pd 
import numpy as np 
import requests
import bs4
from bs4 import BeautifulSoup
import re

def tagStringDealing(string):
    NewString = string.replace("\n",'')
    NewString = NewString.replace("\r",'')
    NewString = NewString.replace(" ","")
    return NewString
              

def getHTMLtext(url):
    try:
        r = requests.get(url)

        r.raise_for_status()
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text,'html.parser')
        return soup
    except:
        return "ERROR"

def DataDealing(HTML):
    Inittag = HTML.find(name = "ul",attrs={"class":"newlist"})
    # print(Inittag.find_all("li"))
    read=[];quote = [];stockID = []; barName = [];NewsTitle = [];date = [];last = []
    for tag in Inittag.find_all("li"):
        try:
            # print(type(tag))
            if isinstance(tag,bs4.element.Tag):
        
                aTags = tag.span.find_all("a")
                href = aTags[0].attrs["href"]
                Stkid = re.search(r"\d{6}",href).group(0)
                stockID.append(Stkid)
                NewsTitle.append(aTags[1].attrs["title"])
                barName.append(aTags[0].string)
                # print(type(tag))
                citeTags = tag.find_all(name = "cite")
                # print(type(citeTags))
                # print(citeTags)
                readNum = citeTags[0].string
                read.append(int(tagStringDealing(readNum)))
                # print(read)
                quoteNum = citeTags[1].string
                quote.append(int(tagStringDealing(quoteNum)))
                date.append(citeTags[3].string)
                last.append(citeTags[4].string)
                
        except:
            pass
    # print(len(date),len(last),len(read),len(quote),len(stockID),len(barName),len(NewsTitle))
    try:
        df = pd.DataFrame(np.array([date,last,read,quote,stockID,barName,NewsTitle]).T,columns = ["日期","最后评论时间","阅读量","评论数","股票代码",'股吧名',"内容"])
        return df
    except:
        return pd.DataFrame()


def main():
    TotalDf = pd.DataFrame()
    for i in range(10):#####调整range的范围可以调整爬取的页面数####
        url = "http://guba.eastmoney.com/default,0_"+str(i)+".html" ####获取股吧前十个page的url来爬虫
        print(i,end=' ')
        soup = getHTMLtext(url)
        df = DataDealing(soup)
        # print(df)
        TotalDf = pd.concat([TotalDf,df],axis = 0)
    print(TotalDf)
    # TotalDf.to_csv("Totaldf.csv")
main()


0 1 2 3 4 5 6 7 8 9        日期       最后评论时间   阅读量 评论数    股票代码            股吧名  \
0   11-21  11-21 21:40   415   4  300026          红日药业吧   
1   11-21  11-21 21:40   322   4  300377           赢时胜吧   
2   11-21  11-21 21:40  2783  16  600336           澳柯玛吧   
3   11-21  11-21 21:40   825  10  300021          大禹节水吧   
4   11-20  11-21 21:40  1700  23  000001          上证指数吧   
..    ...          ...   ...  ..     ...            ...   
78  11-20  11-21 21:35   316   5  000001          上证指数吧   
79  11-20  11-21 21:35   451  11  000001          上证指数吧   
80  11-04  11-21 21:35   748   9  000001          上证指数吧   
81  11-21  11-21 21:35    42   0  161131  易方达战略配售(LOF)吧   
82  11-20  11-21 21:35   843  21  605066          天正电气吧   

                                          内容  
0   这里争论很热闹，其实最近这五天，庄家在反复试探杀跌，但都有强力资金抢筹，这间接的  
1              准备好资金星期一融券做空、依靠糖球的减持影响力做空吃肉……  
2   国人有14亿人口，每人注射两剂疫苗，就是28亿剂，疫苗抗体只能维持6个月左右，一  
3                       我说23号有又粗又长的20厘米有人信吗？  
4   当前市场，仍然处于震荡折腾，压榨廉价筹码阶段$东岳硅材(SZ30